In [1]:
!gdown --id 1eSo7XcOuW1BlLLt4YT5Vo-gK1FgR8xMU

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1eSo7XcOuW1BlLLt4YT5Vo-gK1FgR8xMU
From (redirected): https://drive.google.com/uc?id=1eSo7XcOuW1BlLLt4YT5Vo-gK1FgR8xMU&confirm=t&uuid=65fbee44-6e0b-4f07-9853-b182a26325be
To: /kaggle/working/nature_12K.zip
100%|██████████████████████████████████████| 3.82G/3.82G [00:41<00:00, 90.9MB/s]


In [2]:
!unzip -q nature_12K.zip

In [9]:
from torch.nn import Sequential, Module, ReLU, Conv2d, Linear, MaxPool2d, LogSoftmax, NLLLoss, Dropout, BatchNorm2d, LeakyReLU, GELU, SELU, Mish, CrossEntropyLoss
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from torch import flatten, float, no_grad
from torch.optim import Adam
import torch
import wandb
import math

In [10]:
def get_data(param, type):
    if(type.lower() == 'train'):
        transform = transforms.Compose([
            transforms.Resize(256),
            transforms.RandomHorizontalFlip(p=0.3),
            transforms.RandomRotation(degrees=12),
            transforms.ColorJitter(),
            transforms.CenterCrop(224),
            transforms.ToTensor(), 
            transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])  
        ])
        tdataset = datasets.ImageFolder(root=param['train_data_dir'], transform=transform)
        total = len(tdataset)
        train_sample = math.ceil(total*(0.8))
        val_sample = total-train_sample
        # print(total, train_sample, val_sample)
        train_dataset, validation_dataset = torch.utils.data.random_split(tdataset, [train_sample, val_sample])
        train_dataloader = DataLoader(train_dataset, batch_size=param['batch_size'], shuffle=True)
        validation_dataloader = DataLoader(validation_dataset, batch_size=param['batch_size'], shuffle=False)
        return train_dataloader, validation_dataloader
    
    else:
        transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(size=224),
            transforms.ToTensor(), 
            transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])  
        ])
        test_dataset = datasets.ImageFolder(root=param['test_data_dir'], transform=transform)
        test_dataloader = DataLoader(test_dataset, batch_size=param['batch_size'])
        return test_dataloader

In [12]:
def train():

    wandb.init()
    param = wandb.config
    wandb.run.name = f'GoogLeNet_strategy_{param.strategy}_batchSz_{param.batch_size}_epochs_{param.epochs}_layersToFreeze_{param.layers_to_freeze}'

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    pmodel = models.googlenet(pretrained=True)

    if(param['strategy'] == 'all_freeze'):
        num_features = pmodel.fc.in_features
        pmodel.fc = Linear(num_features, 10)
        for name, par in pmodel.named_parameters():
            if name not in ['fc.weight', 'fc.bias']:
                par.requires_grad = False

    elif(param['strategy'] == 'k_freeze'):
        layers_to_freeze = list(pmodel.children())[:param['layers_to_freeze']]
        for x in layers_to_freeze:
            for y in x.parameters():
                y.requires_grad = False
        num_features = pmodel.fc.in_features
        pmodel.fc = Linear(num_features, 10)
    
    else:
        num_features = pmodel.fc.in_features
        pmodel.fc = Linear(num_features, 10)
    
    
    pmodel = pmodel.to(device)
    optimizer = Adam(pmodel.parameters())
    loss_function = CrossEntropyLoss()
    train_data_loader, validation_data_loader = get_data(param, 'train')

    for epo in range(param['epochs']):
        totalTrainLoss = 0
        totalValLoss = 0
        trainCorrect = 0
        valCorrect = 0
        train_counter=0
        validation_counter=0
        pmodel.train()
        for (image, label) in train_data_loader:
            (image, label) = (image.to(device), label.to(device))
            prediction = pmodel(image)
            loss = loss_function(prediction, label)
            loss.backward()
            optimizer.step()
            totalTrainLoss += loss
            trainCorrect += (prediction.argmax(1) == label).type(float).sum().item()
            train_counter+=1

        ## Validation
        pmodel.eval()
        with no_grad():
            for (image, label) in validation_data_loader:
                (image, label) = (image.to(device), label.to(device))
                pred = pmodel(image)
                loss = loss_function(pred, label)
                totalValLoss += loss
                valCorrect += (pred.argmax(1) == label).type(float).sum().item()
                validation_counter += 1


        tr_ls = (totalTrainLoss/train_counter).cpu().detach().numpy()
        tr_acc = trainCorrect/len(train_data_loader.dataset)
        val_ls = (totalValLoss/validation_counter).cpu().detach().numpy()
        val_acc = valCorrect/len(validation_data_loader.dataset)


        print(f"Epoch --> {epo}")
        print(f"Train Loss --> {tr_ls}")
        print(f"Train Accuracy --> {tr_acc}")
        print(f"Validation Loss --> {val_ls}")
        print(f"Validation Accuracy --> {val_acc}")
        print("-----------------------------------------------------------")
        
        lg={
            'epoch': epo+1,
            'tr_accuracy': tr_acc,
            'val_accuracy': val_acc,
            'tr_loss': tr_ls,
            'val_loss': val_ls
        }
        wandb.log(lg)


    

In [17]:
sweep_config = {
  "method": "grid",
  "name": "PartB GoogLeNet Sweep",
  "metric": {"goal": "maximize", "name": "val_accuracy"},
  "parameters": {
    "batch_size":{"values": [32]},
    "epochs":{"values": [10]},
    "strategy":{"values": ['all_freeze','no_freeze']},  ## K freeze, No Freeze, all_freeze(except last)
    "layers_to_freeze": {"values": [15]},
    "train_data_dir":{"values": ["/kaggle/working/inaturalist_12K/train"]},
    "test_data_dir":{"values": ["/kaggle/working/inaturalist_12K/val"]}
  }
}

In [14]:
wandb.login(key="b8d44a4abbab8753e976a6e5ab717fd669ba99a2")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs24m030 (cs24m030-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [18]:
# wandb.init()
sweep_id = wandb.sweep(sweep_config, project="DA6401_A2")

Create sweep with ID: 4gvvnxb9
Sweep URL: https://wandb.ai/cs24m030-indian-institute-of-technology-madras/DA6401_A2/sweeps/4gvvnxb9


In [19]:
wandb.agent(sweep_id, function=train, count=2)
# wandb.agent(yb8u4pax, function=train, count=2)
wandb.finish()

wandb: Agent Starting Run: 50lba11z with config:
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	layers_to_freeze: 15
wandb: 	strategy: all_freeze
wandb: 	test_data_dir: /kaggle/working/inaturalist_12K/val
wandb: 	train_data_dir: /kaggle/working/inaturalist_12K/train


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch --> 0
Train Loss --> 10.104148864746094
Train Accuracy --> 0.27425
Validation Loss --> 20.360363006591797
Validation Accuracy --> 0.24712356178089045
-----------------------------------------------------------
Epoch --> 1
Train Loss --> 24.344757080078125
Train Accuracy --> 0.2995
Validation Loss --> 24.27935218811035
Validation Accuracy --> 0.36868434217108553
-----------------------------------------------------------
Epoch --> 2
Train Loss --> 24.09234619140625
Train Accuracy --> 0.386625
Validation Loss --> 18.206615447998047
Validation Accuracy --> 0.46473236618309155
-----------------------------------------------------------
Epoch --> 3
Train Loss --> 25.397260665893555
Train Accuracy --> 0.45325
Validation Loss --> 21.937347412109375
Validation Accuracy --> 0.49374687343671836
-----------------------------------------------------------
Epoch --> 4
Train Loss --> 21.812896728515625
Train Accuracy --> 0.493125
Validation Loss --> 21.76420783996582
Validation Accuracy --> 0.

epoch,▁▂▃▃▄▅▆▆▇█
tr_accuracy,▁▁▃▄▅▆▇▇▇█
tr_loss,▁█▇█▆▇▇▆█▆
val_accuracy,▁▃▄▅▆▆▆▇▇█
val_loss,▃▆▁▄▄▇▄▄█▁
epoch,10
tr_accuracy,0.6445
tr_loss,21.05475
val_accuracy,0.68584
val_loss,18.21377


wandb: Agent Starting Run: esut1xjv with config:
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	layers_to_freeze: 15
wandb: 	strategy: no_freeze
wandb: 	test_data_dir: /kaggle/working/inaturalist_12K/val
wandb: 	train_data_dir: /kaggle/working/inaturalist_12K/train


Epoch --> 0
Train Loss --> 2.8941736221313477
Train Accuracy --> 0.12925
Validation Loss --> 20.316925048828125
Validation Accuracy --> 0.09354677338669334
-----------------------------------------------------------
Epoch --> 1
Train Loss --> 2.374051570892334
Train Accuracy --> 0.100625
Validation Loss --> 2.393130302429199
Validation Accuracy --> 0.10655327663831916
-----------------------------------------------------------
Epoch --> 2
Train Loss --> 2.3056390285491943
Train Accuracy --> 0.1145
Validation Loss --> 2.2871947288513184
Validation Accuracy --> 0.15257628814407204
-----------------------------------------------------------
Epoch --> 3
Train Loss --> 2.276890277862549
Train Accuracy --> 0.14175
Validation Loss --> 2.3211631774902344
Validation Accuracy --> 0.1500750375187594
-----------------------------------------------------------
Epoch --> 4
Train Loss --> 2.2722673416137695
Train Accuracy --> 0.14575
Validation Loss --> 2.335256576538086
Validation Accuracy --> 0.163

epoch,▁▂▃▃▄▅▆▆▇█
tr_accuracy,▄▁▂▅▅▇▇▇██
tr_loss,█▃▂▂▂▁▁▁▁▁
val_accuracy,▁▂▅▅▅▆▆▇▅█
val_loss,█▁▁▁▁▁▁▁▁▂
epoch,10
tr_accuracy,0.17663
tr_loss,2.21463
val_accuracy,0.2051
val_loss,4.64751
